In [13]:
%load_ext autoreload
%autoreload 2

from mint.model.transformer import Transformer, TransformerConfig
from mint.common import create_config, to_dict
from mint.trainer import Trainer, TrainerConfig
import os
from mint.translator import Translator
from mint.dataset import Dataset
from mint.tokenizer import Tokenizer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
DATASET_PATH = "../datasets/en_sk/"

dataset = Dataset.load_preprocessed(DATASET_PATH)
source_tokenizer = Tokenizer.load("../datasets/en_sk/source_tokenizer/")
target_tokenizer = Tokenizer.load("../datasets/en_sk/target_tokenizer/")
source_tokenizer.pad_token = "<pad>"
target_tokenizer.pad_token = "<pad>"

[[494, 494, 494, 79, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000]]


In [17]:
config = create_config(TransformerConfig())
config.glob.d_model = 512
config.glob.n_heads = 8
config.glob.max_seq_len = 128
config.glob.d_feedforward = 2048
config.glob.p_dropout = 0.1

config.encoder_config.n_blocks = 10
config.encoder_config.vocab_size = 10000
config.decoder_config.n_blocks = 10
config.decoder_config.vocab_size = 10000

model = Transformer(config)

print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")

{'n_blocks': 10, 'vocab_size': 10000, 'transformer_block_config': {'d_model': 512, 'd_feedforward': 2048, 'p_dropout': 0.1, 'attention_config': {'n_heads': 8, 'd_model': 512, 'max_seq_len': 128, 'context_window': None}}, 'embedding_config': {'vocab_size': 10000, 'd_model': 512, 'max_seq_len': 128, 'learnable_positional_embeddings': True}}
Number of parameters: 126471680


In [18]:
trainer_config = create_config(TrainerConfig())
trainer_config.logger_config.experiment_name = "exp2"
trainer_config.warmup_steps = 3000
trainer_config.learning_rate = 1e-4
trainer = Trainer(model, dataset, **to_dict(trainer_config), source_tokenizer=source_tokenizer, target_tokenizer=target_tokenizer)
trainer.train(10)

Train 1/10 avg loss: 9.2853:   0%|          | 11/28140 [00:03<2:48:20,  2.78it/s]


KeyboardInterrupt: 

In [17]:
translator = Translator(model, source_tokenizer, target_tokenizer)

translator.translate("Good evening", max_length=128)

['Dobré večera<|endoftext|>']